# Corona Health - Psychological Health of adults ('Parents') Study

## Purpose of this Notebook
- [ ] Clean the dataset and save cleaned version
- [x] Get an statistical overview
    - [x] How many users?
    - [x] How many assessments (= filled out questionnaires)?
    - [x] Date range of the dataset?
    - [x] User-assessment distribution
- [ ] Potential target for classification?
- [ ] Potential features for classification?
    
    

#### Statistical overview

In [7]:
# imports
import pandas as pd
from datetime import date, datetime

In [8]:
# to import own modules, a workaround - assuming the curent working directory is the folder (your/local/path/UsAs/src/d01_analysis
import sys
sys.path.insert(0, "../..")

from src.d00_utils import helpers

In [9]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/ch/22-07-01_rki_parent_followup.csv')

In [10]:
# create result dict to save to disk
result = dict()

In [11]:
# Number of unique users
print('Number of user:\t', df.user_id.nunique())
result['n_users'] = df.user_id.nunique()

Number of user:	 1428


In [12]:
# number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 9957


In [13]:
# date range
form = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, form)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, form)



print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)


result['Time range in days'] = delta.days

Start:	 2020-07-21 14:13:47
End:	 2022-02-03 17:30:12

Date Range in
Years:	 1.5397260273972602
Months:	 46.833333333333336
Days:	 562


#### Distribution of filled out questionnaires

In [14]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
ser
result.update(dict(ser))

### Average period length between two filled out questionnaires

In [15]:
# res = result
res = helpers.find_mode_period_length(df, form='%Y-%m-%d %H:%M:%S', date_col_name='created_at')
result.update(res)

### Statistical overview

In [16]:
result

{'n_users': 1428,
 'n_assessments': 9957,
 'First assessment from': '2020-07-21 14:13:47',
 'Latest assessment from': '2022-02-03 17:30:12',
 'Time range in days': 562,
 Interval(0, 1, closed='right'): 439,
 Interval(1, 2, closed='right'): 224,
 Interval(2, 3, closed='right'): 137,
 Interval(3, 5, closed='right'): 174,
 Interval(5, 10, closed='right'): 207,
 Interval(10, 100, closed='right'): 247,
 Interval(100, 1000, closed='right'): 0,
 'avg hours between two assessments': 163.84839310829818,
 'avg days between two assessments': 6.827016379512424,
 'std_hours': 51.87054059436013,
 'std_days': 2.161272524765005}

In [17]:
# read in codebook and reduce to columns and rows of interest
cb = pd.read_excel('../../data/d00_helpers/codebook/ch/rki_parent.xlsx', sheet_name='FollowUp', header=4)
cb = cb[cb.elementtype=='question']
cb = cb.iloc[:, :30]

C:\Users\joa24jm\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
